# Welcom to HiPIMS!


This document gives you a guidance and embeds the code to run the hipims code for case study.
You can run this tutorial in a couple of ways:

**In the cloud:** This is an easy way to get started! Run in Google Colab link, which opens an integrated notebook in Google Colab, with the code in a fully-hosted environment.

**Locally:** This option requires you to setup HiPIMS first on your local machine (installation instructions). Download the notebook or copy the code into your favorite IDE.

## About
HiPIMS names for High-Performance Integrated hydrodynamic Modelling System. It uses state-of-art numerical schemes (Godunov-type finite volume) to solve the 2D shallow water equations for flood simulations. To support high-resolution flood simulations, HiPIMS is implemented on multiple GPUs (Graphics Processing Unit) using CUDA/C++ languages to achieve high-performance computing. To find out how to use the model, please see the wiki.

## 1. Install and import the package 

In [ ]:
!pip install hipims2

In [7]:
from hipims.InputHipims import InputHipims
from hipims import flood

## 2. Data Formats and Types
To run the flood model, you need to prepare specific data types. Below are the minimum required data and additional **data types** that can enhance your simulations.

1. **Spatial Distribution Data (SDD)**:  
   These data describe spatial patterns and variations across different locations. The supported formats include:  
   - **Single Value**: Represents a uniform value across the entire spatial domain, used when the phenomenon is consistent across all locations.
   - **`np.array` (NumPy Array)**: Allows users to input customised values as an array to represent spatial data.  
     *Note: The shape of the `np.array` must match the shape of the DEM to maintain spatial consistency.*
   - **`tif` (GeoTIFF) File**: A raster data format that includes geographic coordinate information, allowing for detailed spatial modelling.  
     *Note: The grid size of the GeoTIFF file must match that of the DEM.*

2. **Time Series Data (TSD)**:  
   These data represent temporal dynamics and support the following formats:  
   - **Single Value**: Represents a uniform value across the entire time period, implying no variation over time.
   - **`np.array` (NumPy Array)**: Allows users to input customised time series data. The first column represents the time series (unit: seconds), and the subsequent columns contain the corresponding values.
   - **File**: Time series data can be stored in a file, such as a `.txt` or `Excel file`.  
     *Note: If the data are on a specific sheet in an Excel file, use the format `['filename', 'sheetname']`. Otherwise, use `'filename'`.*

## 3. Running HiPIMS for Catchment Case Study
3.1 & 3.9 are necessary. 

3.2-3.8 are optional, call only differ from default.

*Note: SDD refers to Spatial Distribution Data; TSD refers to Time Series Data.*


### 3.1 Quickstart

To define the model case, use the following:

- **Data Requirements**:
  - `DEM.tif` : Digital Elevation Model (DEM), providing terrain elevation data necessary for calculating water flow paths.

- **Function**: `InputHipims`

  **Keyword Arguments**:

<div align="center">

| Argument           | Type    | Description                                                               |
|--------------------|---------|---------------------------------------------------------------------------|
| `case_folder`      | String  | Path to the folder containing the case files.                              |
| `dem_path`         | String  | Path to the Digital Elevation Model (DEM) file, providing terrain data.    |
| `output_folder`    | String  | Path to the folder where the simulation output files will be saved.        |
| `rainfall_source`  | TSD     | Source of the rainfall data.                                               |

</div>


In [5]:
# minimal requirements
case_folder = 'HiPIMS_2024/sample'
dem_path = 'DEM.tif'
output_folder = 'output'
rainfall_source = 0

inputs = InputHipims(case_folder = case_folder,
                     dem_path = dem_path,
                     output_folder= output_folder,
                     rainfall_source = rainfall_source)

### 3.2. Set Up Model Parameters

Define the model parameters to control the simulation's configuration. 

- **Function**: `set_model_parameters`

  **Keyword Arguments**:

<div align="center">

| Argument              | Type     | Default  | Description                                                            |
|-----------------------|----------|----------|------------------------------------------------------------------------|
| `num_GPU`             | Integer  | `1`      | Number of GPU devices to use.                                          |
| `start_time`          | Numeric  | `0`      | Simulation start time in seconds.                                      |
| `GPU_device_ID`       | Integer  | `0`      | ID of the GPU device.                                                  |
| `end_time`            | Numeric  | `3600`   | Total simulation time in seconds.                                      |
| `output_interval`     | Numeric  | `600`    | Specifies how frequently (in seconds) the output files are saved.      |
| `second_order`        | Boolean  | `False`  | Specifies whether to use a second-order solver.                        |
| `projected_coordinate`| Boolean  | `False`  | Indicates whether the geographic coordinate is in a projected coordinate system.                        |

</div>


In [39]:
inputs.set_model_parameters(end_time=12000)

### 3.3 Set Initial Condition

Define the initial conditions for the simulation. By default, the simulation area is considered completely dry.
- **Function**: `set_initial_condition`

  **Keyword Arguments:**

<div align="center">

| Argument     | Type  | Unit       | Default | Description                                |
|--------------|-------|------------|---------|--------------------------------------------|
| `h0`         | SPD   | Metres (m) | `0`     | Initial water depth.                       |
| `hU0x`       | SPD   | Metres (m) | `0`     | Initial unit discharge in the x-direction. |
| `hU0`        | SPD   | Metres (m) | `0`     | Initial unit discharge in the y-direction. |

</div>



In [4]:
inputs.set_initial_condition(h0='depth.tif',
                              hU0x='hux.tif',
                              hU0y='huy.tif')

### 3.4. Set Rainfall
Define the rainfall data for the simulation. By default, there is no rainfall.

- **Function**: `set_rainfall`

  **Keyword Arguments:**

<div align="center">

| Argument      | Type                             | Unit                     | Default | Description                                                                                           |
|---------------|----------------------------------|--------------------------|---------|-------------------------------------------------------------------------------------------------------|
| `rain_mask`   | SPD   | N/A     | `0`     | A mask that defines the rainfall station indices. indices.                                                     |
| `rain_source` | TSD   | mm/hr   | `0`     | The source of rainfall data. The first column represents time, and the subsequent columns correspond to the indices in `rain_mask`, starting from 0. |

</div>


In [5]:
inputs.set_rainfall(rain_mask=0,rain_source='rainfall_source.xlsx')

### 3.5. Set Boundary Condition and Show Domain Map

Define the Input/Output (IO) boundaries at the edges of the model domain. These boundaries typically represent river inflow from upstream or river water depth/tidal levels downstream.

- **Function**: `set_boundary_condition`

  **Keyword Arguments:**

| Argument           | Type                   | Description                                                                                                    |
|--------------------|------------------------|----------------------------------------------------------------------------------------------------------------|
| `bound_list`       | List of dictionaries   | A list where each dictionary represents a boundary condition for the model.                                    |
| - `extent`         | Array                  | Specifies the boundary's spatial extent using minimum and maximum X and Y coordinates in the format `[x_min, x_max, y_min, y_max]`. |
| - `type`           | String                 | Defines the type of boundary condition (see **Boundary Condition Types** below for details).                   |
| - `source`         | TSD / None             | Specifies the source of data for the boundary condition. (see **Boundary Condition Types** below for details). |
| `outline_boundary` | String                 | Specifies the condition applied to the model's outer boundary.                                                 |

- **Boundary Condition Types**

  The following types of boundary conditions are available:

<div align="center">

| Boundary Type   | Description                                                                 | Source | Unit                        |
|-----------------|-----------------------------------------------------------------------------|--------------------------------------|------------------------------|
| `RIGID`         | A solid, non-movable boundary.                                               | None | N/A                          |
| `WALL_SLIP`     | Allows slip conditions along a wall boundary.                                | None | N/A                          |
| `OPEN`          | An open boundary where flow can freely enter or exit.                        | None  | N/A                          |
| `H_GIVEN`       | A boundary with a specified water depth.                                     | TSD          | m                |
| `Q_GIVEN`       | A boundary with a specified flow discharge.                                  | TSD| m³/s|
| `WL_GIVEN`      | A boundary with a specified water level.                                     | TSD| m               |
| `FALL`          | A specific condition, such as a falling water level.                         | None           | N/A |

</div>

In [6]:
boundary1 = {'extent': [339593, 339629, 554243, 554266],
             'type': 'Q_GIVEN', 
             'source': ['Q_source.xlsx', 'Cummersdale']}
boundary2 = {'extent': [346948, 347033, 554233, 554272],
             'type': 'Q_GIVEN', 
             'source': ['Q_source.xlsx', 'Great Corby']}
boundary3 = {'extent': [349002, 349030, 558367, 558424],
             'type': 'Q_GIVEN', 
             'source': ['Q_source.xlsx', 'Green Holme']}
boundary4 = {'extent': [341185, 341242, 554236, 554558],
             'type': 'Q_GIVEN', 
             'source': ['Q_source.xlsx', 'Harraby Green Business Park']}
bound_list = [boundary1, boundary2, boundary3, boundary4]

inputs.set_boundary_condition(bound_list=bound_list,
                              outline_boundary = 'OPEN')

### 3.6. Land Cover and Related Processes
Land cover types, such as forests, urban areas, or fields, affect the speed and volume of surface water flow by influencing processes like surface friction, infiltration, and sewer drainage.

#### 3.6.1 Defining Land Cover
Define the land cover data for the simulation to represent different types of land use within the case domain.

- **Function**: `set_landuse` 

    **Keyword Argument:**

<div align="center">

| Argument        | Type                             | Default | Description                                                |
|-----------------|----------------------------------|---------|------------------------------------------------------------|
| `landuse_mask`  | SPD (Spatial Distribution Data)  | `0`     | Indexes the land use distribution within the case domain.   |

</div>

In [7]:
inputs.set_landuse(landuse_mask='Landcover.tif')

#### 3.6.2 Common Data Formats for Land Cover-Based Parameters
All land cover-based parameters share the following data formats:
- **Single Value**:  A numeric value applied for all land use categories.
- **`np.array` (NumPy Array)**: An array of numeric values corresponding to the land use categories defined in the `landuse_mask`. The values should be ordered from the smallest to the largest land use indices.
- **List**:  A list format that allows for custom settings:
   - **`'default_value'`**: The default parameter value for all land use categories unless specified otherwise.
   - **`'special_land_type_value'`**: The index of a specific land use category requiring a different parameter value.
   - **`'special_param_value'`**: The value corresponding to the `special_land_type_value` for the specified parameter.

#### 3.6.3 Friction
Friction in hydrodynamic modelling refers to the resistance water encounters as it flows over different surfaces, such as riverbeds, floodplains, or urban landscapes. This resistance is primarily influenced by the roughness of the surface, which slows down the water and dissipates its energy.

- **Function**: `set_manning` 

    **Keyword Argument:**

<div align="center">

| Argument        | Type                             | Default | Description                                                |
|-----------------|----------------------------------|---------|------------------------------------------------------------|
| `manning`  | Land Cover-Based Parameters  | `0`     | The roughness coefficient. Higher values represent rougher surfaces, slowing down water flow.  |

</div>


In [8]:
inputs.set_manning(manning={'default_value': 0.035,
                            'special_land_type_value': [1],
                            'special_param_value': [0.03]})

#### 3.6.4 Infiltration

Infiltration is the process by which water on the ground surface enters the soil. In HiPIMS, the [**Green-Ampt model**](https://en.wikipedia.org/wiki/Green%E2%80%93Ampt_model) is used to calculate the amount of water that infiltrates into the soil over time.

There are two methods for setting up infiltration. 

***!!! Run one of the following methods depending on your own case:***



##### Method 1: Manual Parameter Setting
Directly assign specific values for each parameter based on different land cover types.

- **Function**: `set_infiltration`

**Keyword Arguments:**

<div align="center">

| Argument                 | Type                         | Default | Description                                                                 |
|--------------------------|------------------------------|---------|-----------------------------------------------------------------------------|
| `cumulative_depth`       | Land Cover-Based Parameters  | `0`     | Cumulative infiltration depth since the start of the simulation (m).         |
| `hydraulic_conductivity` | Land Cover-Based Parameters  | `0`     | The rate at which water moves through the soil, depending on soil properties (m/s). |
| `capillary_head`         | Land Cover-Based Parameters  | `0`     | Soil suction at the wetting front, affecting water infiltration (m).         |
| `water_content_diff`     | Land Cover-Based Parameters  | `0`     | Difference in water content between saturated and initial conditions (m³/m³). |

</div>

In [17]:
inputs.set_infiltration(cumulative_depth=0)


##### Method 2: Automatic Parameter Setting by Soil Type
Automatically determine parameter values based on the soil type associated with each land cover type.

Soil types are based on the USDA Soil Classification: Clay, Silty Clay, Sandy Clay, Clay Loam, Silty Clay Loam, Sandy Clay Loam, Silt Loam, Loam, Sandy Loam, Loamy Sand, Sand, Impervious Surface, Water.

- **Function**: `set_soil_type`

    **Keyword Arguments:**

<div align="center">

| Argument | Format | Description |
|----------|--------|-------------|
| `soil_type`   | List | Specifies the soil type settings for different land use categories.     |
| - `default_soil_type` | String | Default soil type for all land use categories unless specified otherwise. |
| - `special_land_type_value` | list of values / None | Index of a specific land use category requiring a different soil type.|
| - `special_soil_type` | list of string / None | The soil type corresponding to the `special_land_type_value`.|
</div>



In [19]:
inputs.set_soil_type(soil_type={'default_soil_type': 'Clay',
                                'special_land_type_value': [1],
                                'special_soil_type': ['Water']})

#### 3.6.5 Sewer System
By defining areas in the model that drain water into the sewer system, users can simulate how water is removed from specific land cover types, such as roads, at a controlled rate.

- **Function**: `set_sewer_sink`

    **Keyword Arguments:**

<div align="center">

| Argument     | Type                        | Default | Description                                                               |
|--------------|-----------------------------|---------|---------------------------------------------------------------------------|
| `sewer_sink` | Land Cover-Based Parameters | `0`     | Specifies the rate at which water is drained into the sewer system (m/s). |

</div>


In [10]:
inputs.set_sewer_sink(sewer_rate=0)

### 3.7 Set river gauges
Set gauges with the river channel to monitor timeseries of water level and velocity

In [11]:
gauge_coords = np.array([[340194, 556641], [342892, 557903]])
inputs.set_gauges_position(gauge_coords)

### 3.8. Check your data before running

In [6]:
print(inputs)

---------------------- Model information ---------------------
* To change paramaeters, use the 'set_model_parameters' function. For details, visit http_path
case_folder : /home/lunet/cvxt2/HiPIMS_2024/sample
num_GPU : 1
GPU_device_ID : 0
total_simulation_time : 3600
output_interval : 600
num_gauges : 0
projected_coordinate : False
dem_path : /home/lunet/cvxt2/HiPIMS_2024/sample/input/DEM.tif


---------------------- Initial condition ---------------------
* To change paramaeters, use the 'set_initial_condition' function. For details, visit http_path
h0 : 0
hU0x : 0
hU0y : 0


---------------------- Boundary condition ---------------------
* To change paramaeters, use the 'set_boundary_condition' function. For details, visit http_path
num_boundary : 0
bound_list : None
outline_boundary : FALL
source_path : None


---------------------- Rainfall ---------------------
* To change paramaeters, use the 'set_rainfall' function. For details, visit http_path
rain_mask : 0
rain_source : 0


--

### 3.9 Run flood model

In [20]:
flood.run(inputs)